In [1]:
import cv2
import numpy as np
import matplotlib.pylab as plt
import glob
import time

### version1

In [2]:
# 필터 커널 세팅
kernel = np.ones((3, 3), np.uint8)
kernel2 = np.ones((5, 5), np.uint8)

# 영상 경로 설정
video_path= './video1.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 영상 시간 (프레임) 세팅
frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) *2 / 3 )
#cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)

# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 240])

red3 = np.array([155, 30, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])



while True :
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
    #####################################  프레임 전처리  #####################################
    # 가우시안 필터 적용         
    blur = cv2.GaussianBlur( img, (3,3), 0 )    
    # HSV 영상으로 변환  
    hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 
    
    # 색상에 따른 마스크 생성
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    
    # 마스크 결합
    mask_red = mask_red1 + mask_red2
    
    # 비트 and 연산
    res_red2 = cv2.bitwise_and(img, img, mask = mask_red)
    
    # 침식 (노이즈제거) -> 팽창(픽셀 살리기) 
    erosion_image = cv2.erode(mask_red, kernel, iterations=1)  
    dilation_image = cv2.dilate(erosion_image, kernel2, iterations=1)  
    
    ########################################  객체 분리  ######################################
    numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(dilation_image)
    ##
    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid)) :
            continue

        x, y, w, h, area = stats[idx]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        if y < 500 and area < 4500 and area > 80 and abs(w-h) < 5 : # if area > 120 and abs(w-h) < 5 : # abs : 정사각형에 가까운 네모
            # 원 검출 영역
            detected_img = img[y : y+h,  x : x+w]
            gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)                 
            # 원 검출
            circles = cv2.HoughCircles(gray_detected_img,
                                       cv2.HOUGH_GRADIENT,
                                       1, 
                                       100,
                                       param1=200,
                                       param2=30, 
                                       minRadius=5, 
                                       maxRadius=40)

            if circles is not None :
                circles = np.uint16(np.around(circles))
                
                for i in circles[0, :] :
                    text2 = "{} , {}".format(centerX, centerY)
                    #cv2.circle(img, ( centerX, centerY ), i[2], (0, 255, 0), 2)
                    cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)

                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 1)
                    cv2.putText( img, text2, ( x , y +80 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                    #cv2.imwrite('images7/{}_{}.jpg'.format(video_name, index), img3[y-8:y+h+8, x-8:x+w+8])

    
    
    ############################   
    cv2.imshow('video_file2', res_red2 )
    
    cv2.imshow(video_file, img)

    # # # # # # # # # # # # # ## # # # #
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\core\src\matrix.cpp:757: error: (-215:Assertion failed) dims <= 2 && step[0] > 0 in function 'cv::Mat::locateROI'


### version 2

In [6]:
# 필터 커널 세팅
kernel = np.ones((3, 3), np.uint8)
kernel2 = np.ones((5, 5), np.uint8)

# 영상 경로 설정
video_path= './video1.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 영상 시간 (프레임) 세팅
frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) *2 / 3 )
#cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)

# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 240])

red3 = np.array([155, 50, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])

# red3 = np.array([165, 50, 0]) # 어두운 빨강(갈색계열)
# red4 = np.array([175, 250, 240])

while True :
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
    #####################################  프레임 전처리  #####################################
    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))

    # 가우시안 필터 적용         
    blur = cv2.GaussianBlur( img, (3,3), 0 )    
    # HSV 영상으로 변환  
    hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 
    
    # 색상에 따른 마스크 생성
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    
    # 마스크 결합
    mask_red = mask_red1 + mask_red2
    # mask_red = cv2.inRange(hsv, red3, red4)
    # 비트 and 연산
    res_red2 = cv2.bitwise_and(img, img, mask = mask_red)
    
    # 침식 (노이즈제거) -> 팽창(픽셀 살리기) 
    erosion_image = cv2.erode(mask_red, kernel, iterations=1)  
    dilation_image = cv2.dilate(erosion_image, kernel2, iterations=1)  
    
    ########################################  객체 분리  ######################################
    numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(dilation_image)
    ##
    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid)) :
            continue

        x, y, w, h, area = stats[idx]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        if y < 500 and area < 4500 and area > 150 and abs(w-h) < 5 : # if area > 120 and abs(w-h) < 5 : # abs : 정사각형에 가까운 네모
            # 원 검출 영역
            detected_img = img[y : y+h,  x : x+w]
            gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)                 
            # 원 검출
            circles = cv2.HoughCircles(gray_detected_img,
                                       cv2.HOUGH_GRADIENT,
                                       1, 
                                       100,
                                       param1=200,
                                       param2=30, 
                                       minRadius=5, 
                                       maxRadius=40)

            if circles is not None :
                circles = np.uint16(np.around(circles))
                
                for i in circles[0, :] :
                    text2 = "{} , {}".format(centerX, centerY)
                    #cv2.circle(img, ( centerX, centerY ), i[2], (0, 255, 0), 2)
                    cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)

                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 1)
                    cv2.putText( img, text2, ( x , y +80 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                    #cv2.imwrite('images7/{}_{}.jpg'.format(video_name, index), img3[y-8:y+h+8, x-8:x+w+8])

    
    
    ############################   
    cv2.imshow('video_file2', mask_red )
    
    cv2.imshow(video_file, img)

    # # # # # # # # # # # # # ## # # # #
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

#### 이미지 테스트

In [64]:
# 필터 커널 세팅
kernel = np.ones((3, 3), np.uint8)
kernel2 = np.ones((5, 5), np.uint8)

video_path= './night/20191024_213234.jpg'

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]


# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 50])

red3 = np.array([155, 50, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])

img = cv2.imread(video_path)
img = cv2.resize(img, (800,1000),cv2.INTER_AREA)

# 가우시안 필터 적용         
blur = cv2.GaussianBlur( img, (3,3), 0 )    
# HSV 영상으로 변환  
hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 

# 색상에 따른 마스크 생성
mask_red1 = cv2.inRange(hsv, red1, red2)
mask_red2 = cv2.inRange(hsv, red3, red4)

# 마스크 결합
mask_red = mask_red1 + mask_red2

# 비트 and 연산
res_red = cv2.bitwise_and(img, img, mask = mask_red)

# 침식 (노이즈제거) -> 팽창(픽셀 살리기) 
erosion_image = cv2.erode(mask_red, kernel, iterations=1)  
dilation_image = cv2.dilate(erosion_image, kernel2, iterations=1)  

########################################  객체 분리  ######################################
numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(dilation_image)
##
for idx, centroid in enumerate(centroids) :
    if stats[idx][0] == 0 and stats[idx][1] == 0 :
        continue
    if np.any(np.isnan(centroid)) :
        continue

    x, y, w, h, area = stats[idx]
    centerX, centerY = int(centroid[0]), int(centroid[1])

    if y < 500 and area < 4500 and area > 80 and abs(w-h) < 10 : 
    # abs : 정사각형에 가까운 네모
    # 원 검출 영역
        detected_img = img[y : y+h,  x : x+w]
        gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)                 
        # 원 검출
        circles = cv2.HoughCircles(gray_detected_img,
                                   cv2.HOUGH_GRADIENT,
                                   1, 
                                   100,
                                   param1=200,
                                   param2=30, 
                                   minRadius=5, 
                                   maxRadius=40)

        if circles is not None :
            circles = np.uint16(np.around(circles))

            for i in circles[0, :] :
                text2 = "{} , {}".format(centerX, centerY)
                #cv2.circle(img, ( centerX, centerY ), i[2], (0, 255, 0), 2)
                cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)

                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 1)
                cv2.putText( img, text2, ( x , y +80 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                
cv2.imshow('video_file3', dilation_image )
cv2.imshow('video_file2', res_red )
cv2.imshow(video_file, img)

# # # # # # # # # # # # # ## # # # #
cv2.waitKey(0)
cv2.destroyAllWindows()